## GENERAL INFORMATION

- In average, a tag is 14 times frequent in the total corpus with a standard deviation of 127 tags.
- There are 28 tags which occurs more than 1000 times in the whole corpus.
- 41% of tags occurs only once in the corpus
- TF-IDF: our X_data has been vectorized to 1130 words from the whole Text + Title corpus (3M+ corpus words).
- TF-IDF: our y_data has been vectorized to 231 most important tags from the 13k+ original tags.
- In average, the MultiOutput(RegressionLogistic) model makes 2 tags identification errors/ mismatch per document (one_zero_loss_score)
- In average, the jaccard score of the model is 0.64 if trained on Text + Title and 0.58 if trained only on Titles - which is a good score. The Text + Title scenario is the most efficient approach.


### Difficultés - pour Camélia

- Ne parvient pas à faire fonctionner la fonction get_best_thresholds(y_true,y_pred), visant à identifier le seuil au dela duquel la prediction d'un tag sera un Faux negatif ou un faux positif. Il est habituellement de 0.5 en classification binaire (o ou 1) mais pour notre cas de figure il est fort probable que ce ne soit pas optimal (voir lien article ci dessous).
- https://github.com/pratiknabriya/NLP-Multilabel-Tags-Prediction-Hackathon/blob/main/Hacklive3_NLP_topic_modeling_hackathon.ipynb
- Je ne parviens pas à faire fonctionner cette fonction => get_best_thresholds(y_cv.values, y_pred_cv_proba_text), cela est du à la difference de shape (l'internaute a des classes et un seul output, nous avons des classes et plusieurs output). Je pense qu'il faille formater par colonne d'output (8400 * 3-4) pour y arriver, mais après des dixaines d'essaies, je ne sais plus vers quoi aller pour avancer.
- Je ne sais pas non plus comment attribuer des tags aux colonnes de la matrice y_cv_proba_text(~8400 * 3 (ou 2 ou 4))

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score, zero_one_loss,balanced_accuracy_score,precision_score, recall_score, hamming_loss

## Chargement du preprocessed DataSet

In [2]:
df = pd.read_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/df_process_text_3.csv')

In [3]:
df

,Title,Body,Tags,_clean_tags,_len_body,Body_2,_len_body_2,Body_3,Title_2
0,giving unix process exclusive rw access directory,way sandbox linux process certain directory gi...,linux ubuntu process sandbox selinux,"['linux', 'ubuntu', 'process', 'sandbox', 'sel...",526,sandbox linux process certain directory give p...,462,sandbox linux process certain directory proces...,giving unix process exclusive rw access directory
1,automatic repaint minimizing window,jframe two panel one panel draw line working m...,java graphics jframe jpanel paint,"['java', 'graphics', 'jframe', 'jpanel', 'paint']",2969,jframe two panel panel draw line working minim...,2855,jframe panel panel draw line minimized window ...,automatic repaint minimizing window
2,man-in-the-middle attack security threat ssh a...,expert network security pardon question smart ...,security ssh ssh-keys openssh man-in-the-middle,"['security', 'ssh', 'ssh-keys', 'openssh', 'ma...",447,expert network security pardon smart automatin...,414,expert network security pardon smart automatin...,man-in-the-middle attack security threat ssh a...
3,managing data access simple winforms app,simple winforms data entry app us sqlite alway...,c# winforms sqlite datatable sqlconnection,"['c#', 'winforms', 'sqlite', 'datatable', 'sql...",2537,simple winforms entry app u sqlite always sing...,2382,winforms entry app sqlite always single-user a...,managing data access winforms app
4,render basic html view,basic node.js app trying get ground using expr...,javascript html node.js mongodb express,"['javascript', 'html', 'node.js', 'mongodb', '...",335,basic node.js app get ground express framework...,292,basic node.js app get ground express framework...,render basic html view
...,...,...,...,...,...,...,...,...,...
49995,bypass vertica error execution time exceeded r...,using ssis tool ole db downloading data vertic...,sql-server ssas oledb sql-server-data-tools ve...,"['sql-server', 'ssas', 'oledb', 'sql-server-da...",206,ssis tool ole db downloading vertica database ...,186,ssis tool ole db downloading vertica database ...,bypass vertica error execution time exceeded r...
49996,conflicting conditional operation currently pr...,using f uploading file already created bucket ...,python amazon-web-services amazon-s boto pytho...,"['python', 'amazon-web-services', 'amazon-s', ...",301,f uploading file already created bucket deleti...,268,uploading file bucket deleting bucket executio...,conflicting conditional operation progress bucket
49997,problem lr_find pytorch fastai course,following jupyter notebook course hit upon err...,python machine-learning deep-learning pytorch ...,"['python', 'machine-learning', 'deep-learning'...",908,jupyter notebook course hit upon line cnn_lear...,781,jupyter notebook course hit upon line cnn_lear...,problem lr_find pytorch fastai course
49998,jsonpatch escape slash jsonpatch+json,json wanted update field process my-process po...,java json rest json-patch http-patch,"['java', 'json', 'rest', 'json-patch', 'http-p...",645,json wanted update field process my-process po...,566,json wanted field process my-process pod some-...,jsonpatch escape slash jsonpatch+json


## Mise en Forme du DATASET

In [4]:
df_clean = df.drop(['Tags','_len_body','Body_2','_len_body_2','Body','Title'], axis=1)
df_clean

,_clean_tags,Body_3,Title_2
0,"['linux', 'ubuntu', 'process', 'sandbox', 'sel...",sandbox linux process certain directory proces...,giving unix process exclusive rw access directory
1,"['java', 'graphics', 'jframe', 'jpanel', 'paint']",jframe panel panel draw line minimized window ...,automatic repaint minimizing window
2,"['security', 'ssh', 'ssh-keys', 'openssh', 'ma...",expert network security pardon smart automatin...,man-in-the-middle attack security threat ssh a...
3,"['c#', 'winforms', 'sqlite', 'datatable', 'sql...",winforms entry app sqlite always single-user a...,managing data access winforms app
4,"['javascript', 'html', 'node.js', 'mongodb', '...",basic node.js app get ground express framework...,render basic html view
...,...,...,...
49995,"['sql-server', 'ssas', 'oledb', 'sql-server-da...",ssis tool ole db downloading vertica database ...,bypass vertica error execution time exceeded r...
49996,"['python', 'amazon-web-services', 'amazon-s', ...",uploading file bucket deleting bucket executio...,conflicting conditional operation progress bucket
49997,"['python', 'machine-learning', 'deep-learning'...",jupyter notebook course hit upon line cnn_lear...,problem lr_find pytorch fastai course
49998,"['java', 'json', 'rest', 'json-patch', 'http-p...",json wanted field process my-process pod some-...,jsonpatch escape slash jsonpatch+json


In [5]:
df_clean['Tags'] = df_clean['_clean_tags']
df_clean = df_clean.drop('_clean_tags', axis=1)

In [6]:
df_clean['Text'] = df_clean['Title_2'] + ' ' + df_clean['Body_3']

In [7]:
df_clean = df_clean.drop('Body_3', axis=1)

In [8]:
df_clean = df_clean.dropna()

In [9]:
df_clean.isna().sum()

Title_2    0
Tags       0
Text       0
dtype: int64

In [10]:
df_clean

,Title_2,Tags,Text
0,giving unix process exclusive rw access directory,"['linux', 'ubuntu', 'process', 'sandbox', 'sel...",giving unix process exclusive rw access direct...
1,automatic repaint minimizing window,"['java', 'graphics', 'jframe', 'jpanel', 'paint']",automatic repaint minimizing window jframe pan...
2,man-in-the-middle attack security threat ssh a...,"['security', 'ssh', 'ssh-keys', 'openssh', 'ma...",man-in-the-middle attack security threat ssh a...
3,managing data access winforms app,"['c#', 'winforms', 'sqlite', 'datatable', 'sql...",managing data access winforms app winforms ent...
4,render basic html view,"['javascript', 'html', 'node.js', 'mongodb', '...",render basic html view basic node.js app get g...
...,...,...,...
49995,bypass vertica error execution time exceeded r...,"['sql-server', 'ssas', 'oledb', 'sql-server-da...",bypass vertica error execution time exceeded r...
49996,conflicting conditional operation progress bucket,"['python', 'amazon-web-services', 'amazon-s', ...",conflicting conditional operation progress buc...
49997,problem lr_find pytorch fastai course,"['python', 'machine-learning', 'deep-learning'...",problem lr_find pytorch fastai course jupyter ...
49998,jsonpatch escape slash jsonpatch+json,"['java', 'json', 'rest', 'json-patch', 'http-p...",jsonpatch escape slash jsonpatch+json json wan...


In [11]:
import nltk
tokenizer = nltk.RegexpTokenizer(r'[a-zA_\-+#]*\.?[a-zA_\+#]+')
doc_tokens = []
for list in df.Tags.values:
    doc_tokens.append(tokenizer.tokenize(list))

In [12]:
doc_tokens = pd.Series(doc_tokens)

In [13]:
test = doc_tokens.apply(len)

In [14]:
test[test > 7]

269      8
581      8
824      9
866      8
966      8
        ..
48521    9
49367    8
49606    9
49684    8
49821    8
Length: 154, dtype: int64

In [15]:
index_to_drop = test[test > 7].index.values

In [16]:
df_clean.shape

(49997, 3)

In [17]:
df_clean = df_clean.drop(labels=index_to_drop, axis=0)

In [18]:
df_clean.shape

(49843, 3)

In [19]:
df_clean

,Title_2,Tags,Text
0,giving unix process exclusive rw access directory,"['linux', 'ubuntu', 'process', 'sandbox', 'sel...",giving unix process exclusive rw access direct...
1,automatic repaint minimizing window,"['java', 'graphics', 'jframe', 'jpanel', 'paint']",automatic repaint minimizing window jframe pan...
2,man-in-the-middle attack security threat ssh a...,"['security', 'ssh', 'ssh-keys', 'openssh', 'ma...",man-in-the-middle attack security threat ssh a...
3,managing data access winforms app,"['c#', 'winforms', 'sqlite', 'datatable', 'sql...",managing data access winforms app winforms ent...
4,render basic html view,"['javascript', 'html', 'node.js', 'mongodb', '...",render basic html view basic node.js app get g...
...,...,...,...
49995,bypass vertica error execution time exceeded r...,"['sql-server', 'ssas', 'oledb', 'sql-server-da...",bypass vertica error execution time exceeded r...
49996,conflicting conditional operation progress bucket,"['python', 'amazon-web-services', 'amazon-s', ...",conflicting conditional operation progress buc...
49997,problem lr_find pytorch fastai course,"['python', 'machine-learning', 'deep-learning'...",problem lr_find pytorch fastai course jupyter ...
49998,jsonpatch escape slash jsonpatch+json,"['java', 'json', 'rest', 'json-patch', 'http-p...",jsonpatch escape slash jsonpatch+json json wan...


In [20]:
df_clean.to_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/df_clean.csv',index= False)

In [21]:
raw_corpus_tags = " ".join(df.Tags.values)
raw_corpus_tags[:100]

'linux ubuntu process sandbox selinux java graphics jframe jpanel paint security ssh ssh-keys openssh'

In [22]:
tokenizer = nltk.RegexpTokenizer(r'[a-zA_\-+#]*\.?[a-zA_\+#]+')
raw_tokens_tags= tokenizer.tokenize(raw_corpus_tags)

In [23]:
tmp_tags = pd.Series(raw_tokens_tags).value_counts()
len(tmp_tags)

18044

In [24]:
tmp_tags[:10]

c#            6699
java          6278
javascript    6236
python        5638
asp.net       4193
php           3366
html          3204
.net          3187
jquery        3042
c++           2938
dtype: int64

In [25]:
np.average(tmp_tags)

14.12835291509643

In [26]:
np.std(tmp_tags)

127.53719247824196

In [27]:
len(tmp_tags[tmp_tags > 1000])

28

In [28]:
tmp_tags[tmp_tags > 150]

c#                   6699
java                 6278
javascript           6236
python               5638
asp.net              4193
                     ... 
gridview              154
dynamic               152
android-fragments     151
codeigniter           151
for-loop              151
Length: 231, dtype: int64

## MultiLabelBinarizer

In [29]:
raw_corpus_tags = " ".join(df.Tags.values)
raw_corpus_tags[:100]

'linux ubuntu process sandbox selinux java graphics jframe jpanel paint security ssh ssh-keys openssh'

In [30]:
tokenizer = nltk.RegexpTokenizer(r'[a-zA_\-+#]*\.?[a-zA_\+#]+')
raw_tokens_tags= tokenizer.tokenize(raw_corpus_tags)

In [31]:
raw_tokens_tags

['linux',
 'ubuntu',
 'process',
 'sandbox',
 'selinux',
 'java',
 'graphics',
 'jframe',
 'jpanel',
 'paint',
 'security',
 'ssh',
 'ssh-keys',
 'openssh',
 'man-in-the-middle',
 'c#',
 'winforms',
 'sqlite',
 'datatable',
 'sqlconnection',
 'javascript',
 'html',
 'node.js',
 'mongodb',
 'express',
 'node.js',
 'express',
 'routes',
 'nested-routes',
 'module-export',
 'c#',
 'asp.net',
 'generics',
 'reflection',
 'webforms',
 'sql',
 'performance',
 'key-value',
 'ignite',
 'in-memory-database',
 'sql',
 'sql-server',
 'sql-server',
 'tsql',
 'query-optimization',
 'linux',
 'posix',
 'exec',
 'fork',
 'wait',
 'c#',
 '.net',
 'ienumerable',
 'overloading',
 'ilist',
 '.net',
 'configuration',
 'import',
 'export',
 'configuration-files',
 'memory',
 'linker',
 'embedded',
 'loader',
 'firmware',
 'ios',
 'regex',
 'swift',
 'string',
 'emoji',
 'http',
 'angular',
 'typescript',
 'request',
 'http-post',
 'sql',
 'function',
 'postgresql',
 'plpgsql',
 'dynamic-sql',
 'python',
 '

In [32]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit([raw_tokens_tags])

MultiLabelBinarizer()

In [33]:
total_wrods_text = mlb.classes_
total_wrods_text

array(['+', '-addin', '-addon', ..., 'zurb-joyride', 'zxing', 'zynq'],
      dtype=object)

### Data formating for TF-IDF

In [35]:
step_1 = df_clean.join(df["Tags"].str.split(' ',6, expand=True))
step_1 = step_1.drop(['Title_2','Tags','Text'],axis=1)
step_2 = step_1.stack()
freq_step_2 = step_2.value_counts()
most_freq_tags = freq_step_2[freq_step_2 > 150].index
least_freq_tags = freq_step_2[freq_step_2 <= 150].index
step_2_df = pd.DataFrame(step_2)
step_2_df = step_2_df[~step_2_df.isin(least_freq_tags.values).any(axis=1)]
step_3 = pd.get_dummies(step_2_df)
step_3.columns = step_3.columns.str.strip('0_')
step_3.columns = step_3.columns.str.strip('-')
step_3.columns = step_3.columns.str.strip('_')
step_4 = step_3.groupby(level=0)
tag_columns = step_4.sum()

/var/folders/lf/xpbzxzn13ss6xxtstbmn6t540000gn/T/ipykernel_32555/270975860.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  step_1 = df_clean.join(df["Tags"].str.split(' ',6, expand=True))


In [36]:
all_tags = tag_columns.astype(bool).sum(axis=0).sort_values(ascending=False)
top_tags = all_tags[all_tags > 100]
top_tag_columns = tag_columns[top_tags.index]

final = pd.concat([df_clean, top_tag_columns], axis=1)
final = final.drop('Tags', axis=1)
final.head(10)

,Title_2,Text,c#,javascript,java,python,php,html,jquery,.net,...,cookies,webpack,dynamic,android-fragments,for-loop,sharepoint,codeigniter,pyqt,jsf,windows-phone
0,giving unix process exclusive rw access directory,giving unix process exclusive rw access direct...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,automatic repaint minimizing window,automatic repaint minimizing window jframe pan...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,man-in-the-middle attack security threat ssh a...,man-in-the-middle attack security threat ssh a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,managing data access winforms app,managing data access winforms app winforms ent...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,render basic html view,render basic html view basic node.js app get g...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,use nested-subfloders routing nodejs express a...,use nested-subfloders routing nodejs express a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,using generic process asp.net form request bus...,using generic process asp.net form request bus...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,select query slow apache ignite,select query slow apache ignite prototyping ap...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,sql server optimizer varying table field alias,sql server optimizer varying table field alias...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,wait process,wait process usual fork exec combination int s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
final = final.dropna()
final.shape

(48433, 233)

## Early data


In [38]:
final.to_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/final.csv', index=False)

In [39]:
train_data, test_data = train_test_split(final, test_size=0.3, random_state=42)

In [40]:
train_data.shape

(33903, 233)

## Preparing vocabulary for TF-IDF

In [41]:
combined_vocab_title = train_data['Title_2'].tolist()

In [42]:
combined_vocab_title[:5]

['web reference asp.net project',
 'angularjs bootstrapui typeahead object service',
 'integrate paypal rest api spring jpa application error code',
 'make wordpress comment checkbox mandatory required posting comment',
 'multiple tab webview']

In [43]:
combined_vocab_text = train_data['Text'].tolist()

In [44]:
combined_vocab_text[1][:100]

'angularjs bootstrapui typeahead object service typeahead fetch list account object service class use'

In [45]:
tags_columns = final.drop(['Title_2','Text'], axis=1)

In [46]:
target_cols = tags_columns.columns.values

print(target_cols[:5])

['c#' 'javascript' 'java' 'python' 'php']


In [47]:
train_data['Text'].iloc[:5]

43036    web reference asp.net project service asmx wcf...
1972     angularjs bootstrapui typeahead object service...
12003    integrate paypal rest api spring jpa applicati...
1241     make wordpress comment checkbox mandatory requ...
49327    multiple tab webview creating tab containing w...
Name: Text, dtype: object

In [48]:
train_data[target_cols][:5]

,c#,javascript,java,python,php,html,jquery,.net,asp.net,android,...,cookies,webpack,dynamic,android-fragments,for-loop,sharepoint,codeigniter,pyqt,jsf,windows-phone
43036,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12003,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1241,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# splitting train data -> train data + validation data
X = train_data['Text']
y = train_data[target_cols]

X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size = 0.25, random_state = 42)
print(X_train.shape, y_train.shape)
print(X_cv.shape, y_cv.shape)

(25427,) (25427, 231)
(8476,) (8476, 231)


## TF-IDF

In [50]:
vectorizer_text = TfidfVectorizer(analyzer= 'word', min_df = 0.009, sublinear_tf = True,token_pattern='[a-zA_][a-zA_\#+-]*')

In [51]:
vectorizer_title = TfidfVectorizer(analyzer= 'word', min_df = 0.009, sublinear_tf = True,token_pattern='[a-zA_][a-zA_\#+-]*')

In [52]:
X_cv.iloc[:5]

26308    load local xml file html ie mobile browser fai...
24946    unable get data server browser opening url int...
35313    set c select option mobile safari chrome text ...
43157    script copy file cd hard disk new directory co...
48402    itemclicklistener fragment recyclerview onclic...
Name: Text, dtype: object

In [53]:
vectorizer_text = vectorizer_text.fit(combined_vocab_text)

X_train_tfidf_text = vectorizer_text.transform(X_train)
X_cv_tfidf_text = vectorizer_text.transform(X_cv)

print(X_train_tfidf_text.shape, y_train.shape)
print(X_cv_tfidf_text.shape, y_cv.shape)

(25427, 1130) (25427, 231)
(8476, 1130) (8476, 231)


In [54]:
total_words_text = vectorizer_text.get_feature_names_out()

In [55]:
total_words_text

array(['__init__', 'a', 'absolute', ..., 'yes', 'yet', 'z'], dtype=object)

In [56]:
vectorizer_title = vectorizer_title.fit(combined_vocab_title)

X_train_tfidf_title = vectorizer_title.transform(X_train)
X_cv_tfidf_title = vectorizer_title.transform(X_cv)

print(X_train_tfidf_title.shape, y_train.shape)
print(X_cv_tfidf_title.shape, y_cv.shape)

(25427, 95) (25427, 231)
(8476, 95) (8476, 231)


In [57]:
total_words_title= vectorizer_title.get_feature_names_out()
total_words_title

array(['access', 'add', 'ajax', 'android', 'angular', 'api', 'app',
       'application', 'array', 'asp', 'button', 'c', 'c#', 'c++', 'call',
       'class', 'code', 'convert', 'create', 'data', 'database',
       'display', 'django', 'element', 'error', 'field', 'file', 'find',
       'form', 'function', 'get', 'google', 'html', 'http', 'image',
       'input', 'io', 'java', 'javascript', 'jquery', 'js', 'json', 'key',
       'library', 'list', 'load', 'make', 'method', 'model', 'multiple',
       'mvc', 'mysql', 'net', 'node', 'object', 'one', 'page', 'php',
       'post', 'problem', 'project', 'property', 'python', 'query',
       'rail', 'request', 'row', 'run', 'script', 'server', 'service',
       'set', 'show', 'spring', 'sql', 'string', 'studio', 'table',
       'test', 'text', 'time', 'type', 'url', 'use', 'user', 'using', 'v',
       'value', 'view', 'way', 'web', 'window', 'work', 'wpf', 'xml'],
      dtype=object)

## TF-IDF word analysis

In [58]:
X_train_idf_test = X_train_tfidf_title.toarray()

In [59]:
index_name = vectorizer_title.get_feature_names_out()
index_name[:5]

array(['access', 'add', 'ajax', 'android', 'angular'], dtype=object)

In [60]:
analyze = vectorizer_title.build_analyzer()
print(analyze(X_train[29520]))

['parse', 'soap', 'response', 'php', 'parsing', 'xml', 'response', 'php', 'get', 'calling', 'web', 'service', 'type', 'parser', 'matter', 'dom', 'sax', 'notice', 'xml', 'file', 'line', 'max', 'simplexml_load_string', 'extract', 'soap', 'envelope', 'space', 'response', 'xml', 'version', 'encoding', 'utf', 'soap', 'envelope', 'xmlns', 'soap', 'http', 'schemas', 'xmlsoap', 'soap', 'envelope', 'xmlns', 'xsi', 'http', 'www', 'w', 'xmlschema-instance', 'xmlns', 'xsd', 'http', 'www', 'w', 'xmlschema', 'soap', 'processrequestresponse', 'xmlns', 'http', 'xxxxxxxxx', 'yyy', 'processrequestresult', 'outputdata', 'lt', 'xml', 'version', 'encoding', 'utf', 'gt', 'lt', 'gt', 'lt', 'record', 'id', 'gt', 'lt', 'course_id', 'gt', 'lt', 'course_id', 'gt', 'lt', 'course_number', 'gt', 'lt', 'course_number', 'gt', 'lt', 'record', 'gt', 'lt', 'record', 'id', 'gt', 'lt', 'course_id', 'gt', 'lt', 'course_id', 'gt', 'lt', 'course_number', 'gt', 'lt', 'course_number', 'gt', 'lt', 'record', 'gt', 'lt', 'gt', 'o

In [61]:
X_train_tfidf_title.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.58474388],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.45211518, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [62]:
# 25,973 docs and 4,285 words
X_train_tfidf_df = pd.DataFrame(X_train_tfidf_title.toarray())

In [63]:
X_train_tfidf_df.columns = index_name
X_train_tfidf_df

,access,add,ajax,android,angular,api,app,application,array,asp,...,using,v,value,view,way,web,window,work,wpf,xml
0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.209666,0.0,0.0,0.0,0.584744
1,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
2,0.452115,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.310695,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.0,0.0,0.0,0.213276,0.541231,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.368444,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25422,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
25423,0.343446,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
25424,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
25425,0.000000,0.000000,0.0,0.0,0.0,0.680410,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [64]:
doc_1 = X_train_tfidf_df.iloc[0]

In [65]:
doc_1_proba = doc_1[doc_1 > 0.0]
doc_1_proba[(doc_1_proba > 0.1)]

file       0.162016
get        0.180594
http       0.557388
php        0.337597
server     0.192812
service    0.216632
type       0.215621
web        0.209666
xml        0.584744
Name: 0, dtype: float64

## MultiOutputClassifier

In [66]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
clf_text = MultiOutputClassifier(LogisticRegression(n_jobs=-1, max_iter= 200),n_jobs=-1)

In [67]:
clf_title = MultiOutputClassifier(LogisticRegression(n_jobs=-1, max_iter= 200),n_jobs=-1)

In [68]:
y_train.shape

(25427, 231)

In [118]:
X_train_tfidf_title

<25427x95 sparse matrix of type '<class 'numpy.float64'>'
	with 202841 stored elements in Compressed Sparse Row format>

In [70]:
clf_title = clf_title.fit(X_train_tfidf_title, y_train)
clf_title

MultiOutputClassifier(estimator=LogisticRegression(max_iter=200, n_jobs=-1),
                      n_jobs=-1)

In [71]:
clf_text = clf_text.fit(X_train_tfidf_text, y_train)
clf_text

MultiOutputClassifier(estimator=LogisticRegression(max_iter=200, n_jobs=-1),
                      n_jobs=-1)

In [72]:
y_pred_train_text = clf_text.predict(X_train_tfidf_text)
y_pred_cv_text = clf_text.predict(X_cv_tfidf_text)

In [73]:
y_pred_train_title = clf_title.predict(X_train_tfidf_title)
y_pred_cv_title = clf_title.predict(X_cv_tfidf_title)

In [74]:
y_pred_cv_text_df = pd.DataFrame(y_pred_cv_text)
y_pred_cv_text_df.columns = y_cv.columns
y_pred_cv_text_df

,c#,javascript,java,python,php,html,jquery,.net,asp.net,android,...,cookies,webpack,dynamic,android-fragments,for-loop,sharepoint,codeigniter,pyqt,jsf,windows-phone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8471,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8472,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
y_pred_cv_title_df = pd.DataFrame(y_pred_cv_title)
y_pred_cv_title_df.columns = y_cv.columns
y_pred_cv_title_df

,c#,javascript,java,python,php,html,jquery,.net,asp.net,android,...,cookies,webpack,dynamic,android-fragments,for-loop,sharepoint,codeigniter,pyqt,jsf,windows-phone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8472,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
y_cv = y_cv.reset_index(drop=True)
y_cv

,c#,javascript,java,python,php,html,jquery,.net,asp.net,android,...,cookies,webpack,dynamic,android-fragments,for-loop,sharepoint,codeigniter,pyqt,jsf,windows-phone
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8471,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8472,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8474,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## EVALUATION de la prediction du model

In [77]:
def print_corpus_word(x_tfidf, index_number, features_name_list, most_important_words=False):

    ''' fonction qui affiche les mots avec le plus d'importance dans un document'''

    df_X_doc = pd.DataFrame(x_tfidf[index_number].toarray())
    df_X_doc.columns = features_name_list
    print('----------- TOTAL CORPUS WORDS ------------')
    for i in df_X_doc.values[0]:
        if i > 0:
            localisation = np.where(df_X_doc.values[0] == i)
            print(f'{df_X_doc.columns.values[localisation[0]]} : {i} ')

    print('----------- MOST IMPORTANT CORPUS WORDS ------------')
    if most_important_words:
        np.max(df_X_doc.values[0])
        for i in df_X_doc.values[0]:
            if i > np.max(df_X_doc.values[0]) * 0.7:
                localisation = np.where(df_X_doc.values[0] == i)
                print( f'most important words - {df_X_doc.columns.values[localisation[0]]} : {i} ')


In [78]:
doc_2_true = y_cv.iloc[199]
doc_2_true[doc_2_true == 1]

java           1.0
spring         1.0
spring-boot    1.0
spring-mvc     1.0
Name: 199, dtype: float64

In [79]:
doc_2_pred = y_pred_cv_text_df.iloc[199]
doc_2_pred[doc_2_pred == 1]

java           1.0
spring         1.0
spring-boot    1.0
Name: 199, dtype: float64

In [80]:
print_corpus_word(X_cv_tfidf_title, 199, total_words_title, most_important_words=True)

----------- TOTAL CORPUS WORDS ------------
['api'] : 0.24462723838249695 
['call'] : 0.15561224827756473 
['class'] : 0.30273079848750917 
['database'] : 0.34710604596387434 
['error'] : 0.2014740875707929 
['field'] : 0.2701649062253744 
['input'] : 0.2773057625044562 
['json'] : 0.14646965543661622 
['multiple'] : 0.14034984833400505 
['post'] : 0.16524151526378264 
['request'] : 0.360662892074575 
['service'] : 0.257890420794852 
['spring'] : 0.1489766150952924 
['string'] : 0.3638885816354308 
['text'] : 0.1500655473242002 
['xml'] : 0.26676663808401285 
----------- MOST IMPORTANT CORPUS WORDS ------------
most important words - ['class'] : 0.30273079848750917 
most important words - ['database'] : 0.34710604596387434 
most important words - ['field'] : 0.2701649062253744 
most important words - ['input'] : 0.2773057625044562 
most important words - ['request'] : 0.360662892074575 
most important words - ['service'] : 0.257890420794852 
most important words - ['string'] : 0.363888

In [81]:
print_corpus_word(X_cv_tfidf_text, 199, total_words_text, most_important_words=True)

----------- TOTAL CORPUS WORDS ------------
['annotation'] : 0.23969945481892183 
['api'] : 0.12990672149129417 
['async'] : 0.1042495170389859 
['boot'] : 0.11832007385636732 
['call'] : 0.06702838515909698 
['class'] : 0.10986308418897046 
['client'] : 0.08084581910095191 
['configuration'] : 0.15214135226033845 
['contains'] : 0.08781743597312221 
['controller'] : 0.1528076588287192 
['database'] : 0.1782866483834885 
['description'] : 0.09873169768362379 
['e'] : 0.12667141825796935 
['email'] : 0.09783980592095465 
['error'] : 0.142581022283478 
['exception'] : 0.13148263470228783 
['exist'] : 0.10675922924627289 
['exists'] : 0.10584020525815173 
['field'] : 0.13876101892871123 
['functionality'] : 0.10924143165026556 
['future'] : 0.12271455093262813 
['handle'] : 0.15356233926079718 
['input'] : 0.1290185812286649 
['int'] : 0.06971450845494502 
['integer'] : 0.10087034212609593 
['j'] : 0.08489058585103339 
['json'] : 0.07939317284186079 
['message'] : 0.17395313569362225 
['m

## Model Metrics

In [82]:
def calculate_scores(y_true_df,y_pred_df, number_of_score_to_print=None,jaccard=True, f1=False, accuracy=False, loss=False, ham_loss=False, precision=False, recall=False):
    if jaccard:
        print('-------------- JACCARD SCORES -------------')
        print('-------------------------------------------')
        jaccard_scores = {}
        for i in range(len(y_true_df)):
            jaccard_scores[i] = jaccard_score(y_true_df.iloc[i], y_pred_df.iloc[i], average='macro')
        print({A:N for (A,N) in [x for x in jaccard_scores.items()][:10]})
        print('---------- MIN JACCARD SCORE ----------')
        print(jaccard_scores[min(jaccard_scores, key=jaccard_scores.get)])
        print('---------- MAX JACCARD SCORE -----------')
        print(jaccard_scores[max(jaccard_scores, key=jaccard_scores.get)])
        print('---------- AVERAGE JACCARD SCORE ---------')
        print(mean([jaccard_scores[key] for key in jaccard_scores]))

    if f1:
        print()
        print('---------------- F1 SCORES ----------------')
        print('-------------------------------------------')
        f1_scores = {}
        for i in range(len(y_true_df)):
            f1_scores[i] = f1_score(y_true_df.iloc[i], y_pred_df.iloc[i], average='macro')
        print({A:N for (A,N) in [x for x in f1_scores.items()][:10]})
        print('---------- MIN F1 SCORE ----------')
        print(f1_scores[min(f1_scores, key=f1_scores.get)])
        print('---------- MAX F1 SCORE ----------')
        print(f1_scores[max(f1_scores, key=f1_scores.get)])
        print('---------- AVERAGE F1 SCORE ----------')
        print(mean([f1_scores[key] for key in f1_scores]))

    if accuracy:
        print()
        print('------------- ACCURACY SCORES -------------')
        print('-------------------------------------------')
        accuracy_scores = {}
        for i in range(len(y_true_df)):
            accuracy_scores[i] = balanced_accuracy_score(y_true_df.iloc[i], y_pred_df.iloc[i])
        print({A:N for (A,N) in [x for x in accuracy_scores.items()][:10]})
        print('---------- MIN ACCURACY SCORE ----------')
        print(accuracy_scores[min(accuracy_scores, key=accuracy_scores.get)])
        print('---------- MAX ACCURACY SCORE ----------')
        print(accuracy_scores[max(accuracy_scores, key=accuracy_scores.get)])
        print('---------- AVERAGE ACCURACY SCORE ----------')
        print(mean([accuracy_scores[key] for key in accuracy_scores]))

    if loss:
        print()
        print('--------------- LOSS SCORES ---------------')
        print('-------------------------------------------')
        loss_scores = {}
        for i in range(len(y_true_df)):
            loss_scores[i] = zero_one_loss(y_true_df.iloc[i], y_pred_df.iloc[i], normalize=False)
        print({A:N for (A,N) in [x for x in loss_scores.items()][:10]})
        print('---------- MIN LOSS SCORE -----------')
        print(loss_scores[min(loss_scores, key=loss_scores.get)])
        print('---------- MAX LOSS SCORE -----------')
        print(loss_scores[max(loss_scores, key=loss_scores.get)])
        print('---------- AVERAGE LOSS SCORE ---------')
        print(mean([loss_scores[key] for key in loss_scores]))

    if ham_loss:
        print()
        print('------------- HAMMING LOSS SCORES -----------')
        print('---------------------------------------------')
        hamming_loss_scores = {}
        for i in range(len(y_true_df)):
            hamming_loss_scores[i] = hamming_loss(y_true_df.iloc[i], y_pred_df.iloc[i])
        print({A:N for (A,N) in [x for x in hamming_loss_scores.items()][:10]})
        print('---------- MIN HAMMING LOSS SCORE ----------')
        print(hamming_loss_scores[min(hamming_loss_scores, key=hamming_loss_scores.get)])
        print('---------- MAX HAMMING LOSS SCORE ----------')
        print(hamming_loss_scores[max(hamming_loss_scores, key=hamming_loss_scores.get)])
        print('--------- AVERAGE HAMMING LOSS SCORE ---------')
        print(mean([hamming_loss_scores[key] for key in hamming_loss_scores]))


    if precision:
        print()
        print('---------------- PRECISION SCORES -----------')
        print('---------------------------------------------')
        precision_scores = {}
        for i in range(len(y_true_df)):
            precision_scores[i] = precision_score(y_true_df.iloc[i], y_pred_df.iloc[i], average='macro')
        print({A:N for (A,N) in [x for x in precision_scores.items()][:10]})
        print('--------------- MIN PRECISION SCORE -----------')
        print(precision_scores[min(precision_scores, key=precision_scores.get)])
        print('--------------- MAX PRECISION SCORE -----------')
        print(precision_scores[max(precision_scores, key=precision_scores.get)])
        print('--------------- AVERAGE PRECISION SCORE ----------')
        print(mean([precision_scores[key] for key in precision_scores]))

        if recall:
            print()
            print('--------------- RECALL SCORES ---------------')
            print('---------------------------------------------')
            recall_scores = {}
            for i in range(len(y_true_df)):
                recall_scores[i] = recall_score(y_true_df.iloc[i], y_pred_df.iloc[i], average='macro')
            print({A:N for (A,N) in [x for x in recall_scores.items()][:10]})
            print('--------------- MIN RECALL SCORE --------------')
            print(recall_scores[min(recall_scores, key=recall_scores.get)])
            print('--------------- MAX RECALL SCORE --------------')
            print(recall_scores[max(recall_scores, key=recall_scores.get)])
            print('--------------- AVERAGE RECALL SCORE -----------')
            print(mean([recall_scores[key] for key in recall_scores]))

In [83]:
calculate_scores(y_cv,y_pred_cv_text_df,10,f1=True,accuracy=True,loss=True, ham_loss=True, precision=True, recall=True)

-------------- JACCARD SCORES -------------
-------------------------------------------
{0: 0.4108695652173913, 1: 1.0, 2: 0.4935064935064935, 3: 0.49783549783549785, 4: 0.7456331877729258, 5: 1.0, 6: 0.7956140350877192, 7: 0.6934497816593886, 8: 0.4971014492753623, 9: 0.8311499272197962}
---------- MIN JACCARD SCORE ----------
0.24567099567099568
---------- MAX JACCARD SCORE -----------
1.0
---------- AVERAGE JACCARD SCORE ---------
0.6340040737165017

---------------- F1 SCORES ----------------
-------------------------------------------
{0: 0.4637426900584795, 1: 1.0, 2: 0.49673202614379086, 3: 0.49891540130151846, 4: 0.831140350877193, 5: 1.0, 6: 0.8727973568281938, 7: 0.7824175824175825, 8: 0.5540999514798641, 9: 0.89890590809628}
---------- MIN F1 SCORE ----------
0.24781659388646288
---------- MAX F1 SCORE ----------
1.0
---------- AVERAGE F1 SCORE ----------
0.6753935134976119

------------- ACCURACY SCORES -------------
-------------------------------------------


/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py

{0: 0.4429760156632403, 1: 1.0, 2: 0.5, 3: 0.5, 4: 0.75, 5: 1.0, 6: 0.8727973568281938, 7: 0.7, 8: 0.5, 9: 0.8333333333333333}
---------- MIN ACCURACY SCORE ----------
0.24780701754385964
---------- MAX ACCURACY SCORE ----------
1.0
---------- AVERAGE ACCURACY SCORE ----------
0.6552272140241893

--------------- LOSS SCORES ---------------
-------------------------------------------
{0: 4, 1: 0, 2: 3, 3: 1, 4: 2, 5: 0, 6: 2, 7: 3, 8: 2, 9: 1}
---------- MIN LOSS SCORE -----------
0
---------- MAX LOSS SCORE -----------
8
---------- AVERAGE LOSS SCORE ---------
2

------------- HAMMING LOSS SCORES -----------
---------------------------------------------
{0: 0.017316017316017316, 1: 0.0, 2: 0.012987012987012988, 3: 0.004329004329004329, 4: 0.008658008658008658, 5: 0.0, 6: 0.008658008658008658, 7: 0.012987012987012988, 8: 0.008658008658008658, 9: 0.004329004329004329}
---------- MIN HAMMING LOSS SCORE ----------
0.0
---------- MAX HAMMING LOSS SCORE ----------
0.03463203463203463
-------

/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

{0: 0.4956331877729257, 1: 1.0, 2: 0.4935064935064935, 3: 0.49783549783549785, 4: 0.9956331877729258, 5: 1.0, 6: 0.8727973568281938, 7: 0.9934497816593886, 8: 0.663768115942029, 9: 0.9978165938864629}
--------------- MIN PRECISION SCORE -----------
0.24567099567099568
--------------- MAX PRECISION SCORE -----------
1.0
--------------- AVERAGE PRECISION SCORE ----------
0.7468435351902936

--------------- RECALL SCORES ---------------
---------------------------------------------


/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda

{0: 0.4429760156632403, 1: 1.0, 2: 0.5, 3: 0.5, 4: 0.75, 5: 1.0, 6: 0.8727973568281938, 7: 0.7, 8: 0.5, 9: 0.8333333333333333}
--------------- MIN RECALL SCORE --------------
0.24780701754385964
--------------- MAX RECALL SCORE --------------
1.0
--------------- AVERAGE RECALL SCORE -----------
0.6527750900758852


In [84]:
calculate_scores(y_cv,y_pred_cv_title_df,10,f1=True,accuracy=True,loss=True, ham_loss=True, precision=True, recall=True)

-------------- JACCARD SCORES -------------
-------------------------------------------
{0: 0.3261183261183261, 1: 1.0, 2: 0.4935064935064935, 3: 0.49783549783549785, 4: 0.7456331877729258, 5: 0.7478260869565218, 6: 0.6934497816593886, 7: 0.591304347826087, 8: 0.4971014492753623, 9: 0.6623188405797101}
---------- MIN JACCARD SCORE ----------
0.24458874458874458
---------- MAX JACCARD SCORE -----------
1.0
---------- AVERAGE JACCARD SCORE ---------
0.5884679521950281

---------------- F1 SCORES ----------------
-------------------------------------------
{0: 0.3296863603209336, 1: 1.0, 2: 0.49673202614379086, 3: 0.49891540130151846, 4: 0.831140350877193, 5: 0.8322440087145969, 6: 0.7824175824175825, 7: 0.6622807017543859, 8: 0.5540999514798641, 9: 0.7478165938864629}
---------- MIN F1 SCORE ----------
0.2472647702407002
---------- MAX F1 SCORE ----------
1.0
---------- AVERAGE F1 SCORE ----------
0.6217846034528837

------------- ACCURACY SCORES -------------
---------------------------

/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py

{0: 0.3318649045521292, 1: 1.0, 2: 0.5, 3: 0.5, 4: 0.75, 5: 0.75, 6: 0.7477973568281938, 7: 0.6, 8: 0.5, 9: 0.6666666666666666}
---------- MIN ACCURACY SCORE ----------
0.24780701754385964
---------- MAX ACCURACY SCORE ----------
1.0
---------- AVERAGE ACCURACY SCORE ----------
0.6037223608716045

--------------- LOSS SCORES ---------------
-------------------------------------------
{0: 5, 1: 0, 2: 3, 3: 1, 4: 2, 5: 1, 6: 3, 7: 4, 8: 2, 9: 2}
---------- MIN LOSS SCORE -----------
0
---------- MAX LOSS SCORE -----------
9
---------- AVERAGE LOSS SCORE ---------
2

------------- HAMMING LOSS SCORES -----------
---------------------------------------------
{0: 0.021645021645021644, 1: 0.0, 2: 0.012987012987012988, 3: 0.004329004329004329, 4: 0.008658008658008658, 5: 0.004329004329004329, 6: 0.012987012987012988, 7: 0.017316017316017316, 8: 0.008658008658008658, 9: 0.008658008658008658}
---------- MIN HAMMING LOSS SCORE ----------
0.0
---------- MAX HAMMING LOSS SCORE ----------
0.0389610

/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

{0: 0.32753623188405795, 1: 1.0, 2: 0.4935064935064935, 3: 0.49783549783549785, 4: 0.9956331877729258, 5: 0.9978260869565218, 6: 0.8289473684210527, 7: 0.991304347826087, 8: 0.663768115942029, 9: 0.9956521739130435}
--------------- MIN PRECISION SCORE -----------
0.24672489082969432
--------------- MAX PRECISION SCORE -----------
1.0
--------------- AVERAGE PRECISION SCORE ----------
0.6851401320728047

--------------- RECALL SCORES ---------------
---------------------------------------------


/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda

{0: 0.3318649045521292, 1: 1.0, 2: 0.5, 3: 0.5, 4: 0.75, 5: 0.75, 6: 0.7477973568281938, 7: 0.6, 8: 0.5, 9: 0.6666666666666666}
--------------- MIN RECALL SCORE --------------
0.24780701754385964
--------------- MAX RECALL SCORE --------------
1.0
--------------- AVERAGE RECALL SCORE -----------
0.6018113371653456


/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/maurelco/miniconda3/envs/tensorflow_2/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## THRESHOLD [Blocage - besoin d'aide]

In [85]:
y_pred_train_proba_text = clf_text.predict_proba(X_train_tfidf_text)
y_pred_cv_proba_text = clf_text.predict_proba(X_cv_tfidf_text)

In [109]:
y_pred_cv_proba_text

[array([[8.68952390e-01, 1.29141729e-01, 1.75210474e-03, 1.53775471e-04],
        [9.83684562e-01, 1.56477943e-02, 6.08193816e-04, 5.94498040e-05],
        [9.63780999e-01, 3.48925528e-02, 1.23871626e-03, 8.77321710e-05],
        ...,
        [9.19485758e-01, 7.81902717e-02, 2.15963660e-03, 1.64333324e-04],
        [9.46380213e-01, 5.19249202e-02, 1.56922232e-03, 1.25644218e-04],
        [8.67554381e-01, 1.29693078e-01, 2.57198165e-03, 1.80559145e-04]]),
 array([[0.89605627, 0.10394373],
        [0.98019526, 0.01980474],
        [0.94765742, 0.05234258],
        ...,
        [0.91729809, 0.08270191],
        [0.96372151, 0.03627849],
        [0.99080059, 0.00919941]]),
 array([[9.85681887e-01, 1.37743471e-02, 4.84581767e-04, 5.91843088e-05],
        [9.64608022e-01, 3.43794294e-02, 8.93883684e-04, 1.18664624e-04],
        [9.92361014e-01, 7.28650559e-03, 3.06429094e-04, 4.60516488e-05],
        ...,
        [9.13948687e-01, 7.70648656e-02, 8.63138695e-03, 3.55060590e-04],
        [9.63

In [110]:
y_pred_cv_proba_text = np.array(y_pred_cv_proba_text)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (231, 8476) + inhomogeneous part.

In [111]:
y_pred_cv_proba_text = np.concatenate(y_pred_cv_proba_text, axis=1)

In [108]:
y_pred_cv_proba_text_df = pd.DataFrame(y_pred_cv_proba_text)
y_pred_cv_proba_text_df.columns = y_cv.columns
y_pred_cv_proba_text_df

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (231, 8476) + inhomogeneous part.

In [878]:
# def get_best_threshold(true, pred):
#     true_df = pd.DataFrame(true)
#     best_thresholds = []
#     f1_scores = []
#     for idx in range(343):
#         print(true_df.iloc[:][idx])
#         print(pred[:][idx])
#         print((pred[:][idx] > 0.55)*1)
#         f1_scores.append(f1_score(true_df.iloc[:][idx], (pred[:][idx] > 0.55) * 1))
#         best_thresh = np.argmax(f1_scores)
#         best_thresholds.append(best_thresh)
#     return best_thresholds

In [972]:
# def get_best_thresholds(true, pred):
#     thresholds = [i/100 for i in range(0,100)]
#     best_thresholds = []
#     for idx in range(231):
#         f1_scores = [f1_score(true[:,idx], (np.argmax(y_pred_cv_proba_text[idx][idx], axis=1) > thresh) * 1, average='macro') for thresh in thresholds]
#         best_thresh = thresholds[np.argmax(f1_scores)]
#         best_thresholds.append(best_thresh)
#     return best_thresholds

In [115]:
def get_best_thresholds(true, pred):
    thresholds = [i/100 for i in range(100)]
    best_thresholds = []
    for idx in range(len(y_cv.columns)):
        print(idx)
        f1_scores = [f1_score(true[:, idx], (pred[:,idx] > thresh) * 1, average='macro') for thresh in thresholds] #f1_score(y_true_df.iloc[i], y_pred_df.iloc[i], average='macro') f1_score(y_true_df.iloc[i], y_pred_df.iloc[i], average='macro')
        best_thresh = thresholds[np.argmax(f1_scores)]
        best_thresholds.append(best_thresh)
    return best_thresholds

In [87]:
#check that the shapes of y_cv.values and y_pred_cv_proba_text match, since f1_score expects the input to have the same shape.
y_cv.values.shape

(8476, 231)

In [91]:
serie_test = pd.Series(y_pred_cv_proba_text)
serie_test.shape

(231,)

In [105]:
y_pred_cv_proba_text

[array([[8.68952390e-01, 1.29141729e-01, 1.75210474e-03, 1.53775471e-04],
        [9.83684562e-01, 1.56477943e-02, 6.08193816e-04, 5.94498040e-05],
        [9.63780999e-01, 3.48925528e-02, 1.23871626e-03, 8.77321710e-05],
        ...,
        [9.19485758e-01, 7.81902717e-02, 2.15963660e-03, 1.64333324e-04],
        [9.46380213e-01, 5.19249202e-02, 1.56922232e-03, 1.25644218e-04],
        [8.67554381e-01, 1.29693078e-01, 2.57198165e-03, 1.80559145e-04]]),
 array([[0.89605627, 0.10394373],
        [0.98019526, 0.01980474],
        [0.94765742, 0.05234258],
        ...,
        [0.91729809, 0.08270191],
        [0.96372151, 0.03627849],
        [0.99080059, 0.00919941]]),
 array([[9.85681887e-01, 1.37743471e-02, 4.84581767e-04, 5.91843088e-05],
        [9.64608022e-01, 3.43794294e-02, 8.93883684e-04, 1.18664624e-04],
        [9.92361014e-01, 7.28650559e-03, 3.06429094e-04, 4.60516488e-05],
        ...,
        [9.13948687e-01, 7.70648656e-02, 8.63138695e-03, 3.55060590e-04],
        [9.63

In [106]:
y_pred_cv_text

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [116]:
get_best_thresholds(y_cv.values, y_pred_cv_text)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230


[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [104]:
f1_score(y_cv[:, 0], (y_pred_cv_proba_text[0] > 0.55) * 1, average='macro')

InvalidIndexError: (slice(None, None, None), 0)

In [996]:
len(y_cv)

8476

In [1001]:
y_cv

,c#,javascript,java,python,php,html,jquery,.net,asp.net,android,...,cookies,webpack,dynamic,android-fragments,for-loop,sharepoint,codeigniter,pyqt,jsf,windows-phone
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8471,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8472,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8474,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1007]:
y_pred_cv_proba_text_df = pd.DataFrame(y_pred_cv_proba_text)
y_pred_cv_proba_text_df.columns = y_cv.columns
y_pred_cv_proba_text_df

/Users/maurelco/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/construction.py:568: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


ValueError: could not broadcast input array from shape (8476,4) into shape (8476,)

In [1002]:
y_cv.iloc[:,0]

0       1.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8471    0.0
8472    0.0
8473    0.0
8474    0.0
8475    0.0
Name: c#, Length: 8476, dtype: float64

In [986]:
len(y_pred_cv_proba_text)

231

In [987]:
len(y_pred_cv_proba_text[0])

8476

In [1004]:
(y_pred_cv_proba_text[0][:] > 0.1)

array([[ True,  True, False, False],
       [ True, False, False, False],
       [ True, False, False, False],
       ...,
       [ True, False, False, False],
       [ True, False, False, False],
       [ True,  True, False, False]])

In [1008]:
y_pred_cv_proba_text[:]

[array([[8.68955230e-01, 1.29138794e-01, 1.75220537e-03, 1.53770305e-04],
        [9.83684695e-01, 1.56476255e-02, 6.08228929e-04, 5.94502969e-05],
        [9.63779970e-01, 3.48935097e-02, 1.23879399e-03, 8.77258285e-05],
        ...,
        [9.19484343e-01, 7.81915876e-02, 2.15973106e-03, 1.64338407e-04],
        [9.46382532e-01, 5.19225751e-02, 1.56925501e-03, 1.25637923e-04],
        [8.67559179e-01, 1.29688249e-01, 2.57201886e-03, 1.80553188e-04]]),
 array([[0.89605627, 0.10394373],
        [0.98019526, 0.01980474],
        [0.94765742, 0.05234258],
        ...,
        [0.91729809, 0.08270191],
        [0.96372151, 0.03627849],
        [0.99080059, 0.00919941]]),
 array([[9.85681887e-01, 1.37743471e-02, 4.84581761e-04, 5.91843099e-05],
        [9.64608022e-01, 3.43794292e-02, 8.93883671e-04, 1.18664626e-04],
        [9.92361014e-01, 7.28650561e-03, 3.06429091e-04, 4.60516496e-05],
        ...,
        [9.13948686e-01, 7.70648661e-02, 8.63138692e-03, 3.55060599e-04],
        [9.63

In [981]:
clf_text.classes_

[array([0., 1., 2., 3.]),
 array([0., 1.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 4.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2.]),
 array([0., 1., 2.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3., 4.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1., 2.]),
 array([0., 1., 2.]),
 array([0., 1.]),
 array([0., 1., 2., 3.]),
 array([0., 1.]),
 array([0., 1., 2.]),
 array([0., 1., 2., 3., 4., 5.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2.]),
 array([0., 1., 2.]),
 array([0., 1., 2.]),
 array([0., 1., 2.]),
 array([0., 1.]),
 array([0., 1., 2., 3., 4.]),
 array([0., 1., 2., 4.]),
 array([0., 1.]),
 array([0., 1., 2.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1., 2., 3., 4.]),
 array([0., 1., 2.]),
 array([0., 1., 2., 3.]),
 array([0., 1., 2., 3.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),
 array([0., 1.]),


In [985]:
clf_text.feature_names_in_

AttributeError: 'MultiOutputClassifier' object has no attribute 'feature_names_in_'

In [971]:
np.argmax(y_pred_cv_proba_text[:][100], axis=1)

array([0, 0, 0, ..., 0, 0, 0])

In [967]:
len(y_cv.values)

8476

In [963]:
import numpy as np
rounded_labels=np.argmax(y_pred_cv_proba_text[40], axis=1)
rounded_labels

array([0, 0, 0, ..., 0, 0, 0])

In [973]:
get_best_thresholds(y_cv.values,y_pred_cv_proba_text)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [919]:
df_tfidf = pd.DataFrame(X_train_tfidf_text[0].T.todense(), index=vectorizer_text.get_feature_names_out(), columns=["TF-IDF"])
df_tfidf

,TF-IDF
__init__,0.0
a,0.0
absolute,0.0
accept,0.0
access,0.0
...,...
y,0.0
year,0.0
yes,0.0
yet,0.0


In [944]:
best_threshold = get_best_thresholds(y_cv.values, y_pred_cv_proba_text)

AttributeError: 'list' object has no attribute 'iloc'

## ANNEXE - RECHERCHES INTERNET ANCIENNE

In [146]:
from sklearn.datasets import make_multilabel_classification

In [ ]:
from sklearn.utils._testing import assert_almost_equal

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_clean.Tags)
Y = multilabel_binarizer.transform(df_clean.Tags)

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df_clean.Text)

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [ ]:
Y

In [ ]:
X_tfidf

In [ ]:
Y

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=9000)
X_tfidf_resampled, Y_tfidf_resampled = ros.fit_resample(X_tfidf, Y)

In [ ]:
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf_resampled, Y_tfidf_resampled, test_size=0.2, random_state=9000)

In [ ]:
y_train_tfidf

In [ ]:
fig = plt.figure(figsize=(20,20))
(ax_test, ax_train) = fig.subplots(ncols=2, nrows=1)
g1 = sns.barplot(x=Y.sum(axis=0), y=multilabel_binarizer.classes_, ax=ax_test)
g2 = sns.barplot(x=y_train_tfidf.sum(axis=0), y=multilabel_binarizer.classes_, ax=ax_train)
g1.set_title("class distribution before resampling")
g2.set_title("class distribution in training set after resampling")

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss

def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test_tfidf)))
    print("Hamming score: {}".format(hamming_score(y_pred, y_test_tfidf)))
    print("---")

In [ ]:
nb_clf = MultinomialNB()
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=6, tol=None)
lr = LogisticRegression()
mn = MultinomialNB()

for classifier in [nb_clf, sgd, lr, mn]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(x_train_tfidf, y_train_tfidf)
    y_pred = clf.predict(x_test_tfidf)
    print_score(y_pred, classifier)

In [ ]:
okList=[]
for cl in df_tags.columns:
    for n in df_tags[cl]:
        okList.append(n)
        okList=list(set(okList))
del(okList[okList.index('0')])
newDF= pd.DataFrame(columns=okList)

for x in range(df_tags.count()[0]):
    someDict={}
for d in okList:
    rowdata=list(df_tags.iloc[x])
    if d in rowdata:
        someDict[d]=1
    else:
        someDict[d]=0

newDF=newDF.append(someDict,ignore_index=True)
newDF

In [ ]:
df_clean = df_clean.drop('Tags', axis=1)

In [ ]:
df_clean

In [ ]:
counts = df.Tags.value_counts()
firstlast = counts[:5].append(counts[-5:])
firstlast.reset_index(name="count")

In [ ]:
num_classes = 100
grouped_tags = df.groupby("Tags").size().reset_index(name='count')
most_common_tags = grouped_tags.nlargest(num_classes, columns="count")
df.Tags_clean = df.Tags.apply(lambda tag : tag if tag in most_common_tags.Tags.values else None)
df.Tags_clean

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df.Tags)
Y = multilabel_binarizer.transform(df.Tags)

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df.Text)

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)